## Instruction fine-tuning of pre-trained model.

In [56]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + "0.0.32.post2" if v == "2.8.0" else "0.0.29.post3"
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
COMET_API_KEY = os.getenv("COMET_API_KEY")

In [57]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 ## Choose Any! We auto support RoPE Scaling Internally
dtype = None          ## None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Amerpe +
load_in_4bit = True   ## Use 4bit quantization to reduce memory usage. Can be false.

In [58]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [59]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = 1024,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [60]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    lora_alpha = 32,
    lora_dropout = 0,
    target_modules = ["q_proj","k_proj","v_proj","up_proj","down_proj","o_proj","gate_proj"]
)

In [61]:
from datasets import load_dataset,concatenate_datasets

In [62]:
dataset1 = load_dataset("mlabonne/llmtwin",split = "train")
dataset2 = load_dataset("mlabonne/FineTome-Alpaca-100k",split="train[:10000]")
dataset = concatenate_datasets([dataset1,dataset2])

In [63]:
dataset.column_names

['instruction', 'output', 'source', 'score']

In [64]:
dataset.shape

(13001, 4)

In [65]:
alpaca_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}
### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token

def format_samples(examples):
  text = []
  for instruction, output in zip(examples["instruction"],examples["output"],strict=False):
    message = alpaca_template.format(instruction,output) + EOS_TOKEN
    text.append(message)
  return {"text": text}

In [66]:
dataset = dataset.map(format_samples,batched=True,remove_columns = dataset.column_names)

Map:   0%|          | 0/13001 [00:00<?, ? examples/s]

In [67]:
dataset = dataset.train_test_split(test_size=0.05)

In [68]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 12350
    })
    test: Dataset({
        features: ['text'],
        num_rows: 651
    })
})

In [72]:
!pip install comet-ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.9/731.9 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.3 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.3.2
    Uninstalling python-box-7.3.2:
      Successfully uninstalled python-box-7.3.2


In [74]:
import comet_ml

experiment = comet_ml.Experiment(
    api_key = COMET_API_KEY,
    project_name = "unsloth-training",
)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: tensorflow, torch, tensorboard, sklearn, keras.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/dev-mangukiya/unsloth-training/f7fab330464242a384c34e9dba8f384f



In [76]:
from trl import SFTConfig, SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False,     ## Can make training 5x faster for short sequences
    args = SFTConfig(
      per_device_train_batch_size = 2,
      gradient_accumulation_steps = 8,
      warmup_steps = 5,
      num_train_epochs = 1,
      max_steps = 60,
      learning_rate = 3e-4,
      logging_steps = 1,
      optim = "adamw_8bit",
      weight_decay = 0.01,
      lr_scheduler_type = "linear",
      seed = 3407,
      output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/12350 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/651 [00:00<?, ? examples/s]

In [77]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,350 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 59,768,832 of 3,880,848,384 (1.54% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: devmangukiya561 (devmangukiya561-nirma-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.084400
2,1.061900
3,0.999300
4,1.280200
5,0.809900
6,0.790000
7,0.860300
8,0.876500
9,0.959600
10,0.703300


## Let's run the model! You can change the instruction and input - leave the output blank!

In [83]:
from transformers import TextStreamer

In [84]:
FastLanguageModel.for_inference(model)  ## Enable native 2x faster inference

message = alpaca_template.format("Write a paragraph to introduce supervised fine-tuning.","")
inputs = tokenizer([message],return_tensors="pt").to("cuda")
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs,streamer=text_streamer,max_new_tokens=256,use_cache=True)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write a paragraph to introduce supervised fine-tuning.
### Response:

Supervised fine-tuning is a technique used to improve the performance of pre-trained models on specific tasks. It involves using labeled data to further train the model, allowing it to learn task-specific features and improve its accuracy. This approach is particularly useful when dealing with complex tasks that require a deep understanding of the data. By leveraging the knowledge gained from pre-training, supervised fine-tuning enables models to achieve better results and adapt to new domains or datasets.
<|endoftext|>


In [86]:
# ✅ Save LoRA after SFT
model.save_pretrained("Phi-3.5-sft-lora")
tokenizer.save_pretrained("Phi-3.5-sft-lora")

# ✅ Push to Hugging Face Hub (optional)
model.push_to_hub("devmangukiya/Phi-3.5-sft-lora", token=HF_TOKEN)
tokenizer.push_to_hub("devmangukiya/Phi-3.5-sft-lora", token=HF_TOKEN)


Saved model to https://huggingface.co/devmangukiya/Phi-3.5-sft-lora
